In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from urllib import request
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import datetime
import time
import requests
import pandas as pd
from tqdm import tqdm

import urllib

### PR TIMES

In [2]:
# いつからの分を取得するか指定する（開始日付 YYYY-MM-DD）
START_DT_STR = '2021-04-01'
# 検索キーワード
SEARCH_WORD = 'ドトール'
# 取得数
ARTICLE_NUM = 100

start_dt = datetime.datetime.strptime(START_DT_STR, '%Y-%m-%d')

#webdriveの設定
options = webdriver.ChromeOptions()
# options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

#webdriverを起動
# driver = webdriver.Chrome('chromedriver',options=options)
service = Service()
driver = webdriver.Chrome(service=service, options=options)


#PR TIMESのトップページを開く
target_url = 'https://prtimes.jp/'   
driver.get(target_url)

In [3]:
#検索欄をクリックする
driver.find_elements(By.XPATH, "/html/body/header/div/div[2]/div/input")[0].click()

In [4]:
#検索バーにキーワードを入れ、クリックする
kensaku = driver.find_elements(By.XPATH, "/html/body/header/div/div[2]/div/input")[0]
kensaku.send_keys(SEARCH_WORD)
kensaku.send_keys(Keys.ENTER)

In [5]:

SEE_MORE_FLAG = False
while not (len(driver.find_elements(By.XPATH, '//*[@id="__next"]/div/div/main/div/div/section/div/article'))>=ARTICLE_NUM)|(SEE_MORE_FLAG):
  # もっと見るを押す
  try:
    driver.find_elements(By.XPATH, '//*[@id="__next"]/div/div/main/div/div/section/div[2]/button')[0].click()
  except:
    SEE_MORE_FLAG=True
    pass
  # 描画のため、2秒間待つ
  # TODO:判定は変える必要あり
  time.sleep(2)
#記事数を出力（進捗確認用）
print(len(driver.find_elements(By.XPATH, '//*[@id="__next"]/div/div/main/div/div/section/div/article')))

120


In [6]:
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

articles = soup.find_all('article')[:ARTICLE_NUM]

#記事情報を格納する配列
records = []

#記事ごとの情報を取得
for article in tqdm(articles):

    relative_href = article.find_all('a')[0]['href']
    url = urljoin(target_url, relative_href)

    #URLを1記事ずつ開く
    r = requests.get(url)
    html = r.text
    soup = BeautifulSoup(html, "html.parser")

    #記事タイトル
    title_elem = soup.find('article').find('h1')
    if title_elem:
        title = title_elem.text
    else:
        title = ""
    #記事サブタイトル
    sub_title_elem = soup.find('article').find('h2')
    if sub_title_elem:
        sub_title = sub_title_elem.text
    else:
        sub_title = ""
    #会社名
    company = article.find_all('a')[1].text
    #記事公開日時をdatetime表記に変換
    str_to_dt = datetime.datetime.strptime(article.find('time').text, '%Y年%m月%d日 %H時%M分')
    article_time_dt = datetime.datetime(str_to_dt.year, str_to_dt.month, str_to_dt.day, str_to_dt.hour, str_to_dt.minute)
    #記事本文
    content = soup.find('article').get_text(separator='\n', strip=True)

    #配列に記事の情報を追加
    records.append({
        'url': url,
        'title': title,
        'sub_title': sub_title,
        'company': company,
        'article_time_dt': article_time_dt,
        'content': content,
    })

    #1秒間待つ
    time.sleep(1)


100%|██████████| 100/100 [02:01<00:00,  1.22s/it]


In [7]:
pd.DataFrame(records).to_csv(f'prtimes_{SEARCH_WORD}_from_{START_DT_STR}.csv', encoding='utf-8-sig')

### スクレイピング

In [8]:
def scraping_by_year(url):
    html = urllib.request.urlopen(url)
    soup = BeautifulSoup(html, "html.parser")

    pressrelease_link_list = [s['href'] for s in soup.find('div', id='content2').find_all('a', class_='pressrelease_link')]
    len(pressrelease_link_list)

    records = []
    for pressrelease_link in tqdm(pressrelease_link_list):
        url = 'https://www.doutor.co.jp/'+pressrelease_link
        html = urllib.request.urlopen(url)
        soup = BeautifulSoup(html, "html.parser")

        date = soup.find('div', id='content2').find('p', class_='datetxt date').text
        sub_title = soup.find('div', id='content2').find('p', class_='typetxt').text
        title = soup.find('div', class_='newspost body').find_all({'p','div'})[0].text
        # title = soup.find('div', style='text-align: center;').text
        content = soup.find('div', class_='newspost body').get_text().replace(title,'')
        # content = soup.find('div', class_='newspost body').find_all({'p','div'})[1].text

        #配列に記事の情報を追加
        records.append({
            'url': url,
            'title': title,
            'sub_title': sub_title,
            'date': date,
            'content': content,
        })

        #1秒間待つ
        time.sleep(1)
    return records


In [9]:
records = []
for YEAR in [2023,2022,2021]:
    url = f'https://www.doutor.co.jp/news/newsrelease/{YEAR}.html'
    records.extend(scraping_by_year(url))

100%|██████████| 44/44 [00:48<00:00,  1.10s/it]


In [10]:
pd.DataFrame(records).reset_index().to_csv('newsrelease_doutor.csv', index=None, encoding='utf-8-sig')